In [ ]:
import preamble

# Constants

In [ ]:
import numpy as np

# Experiment
USER_ID = 0

# Trajectory Planner
REFERENCE_TRAJECTORY_CSV_PATH = '../data/reference_path.csv'
INIT_POSE = np.array([-0.68769672,  0.09795882, -0.19265416 + 0.02, -1.16, -1.19, 1.18]) # Also used for Robot

# Verbal PD
K_v = 12.3 * 2.0
B_v = 11.4 * 2.0

# F2L
T_proj = 4
N_proj = 256
dt_proj = T_proj / N_proj

# Physical Admittance
M = 2
B = 17.7

# F2L
F2L_MODEL_PATH = '../data/F2L_models/svm_knn.pkl'
FINAL_IMPULSE_THRESHOLD = 0

# Real time data collection
TABLE_COLUMN_NAMES = [
    't',
    'x',
    'x_dot',
    'F_h',
    'x_ref',
    'x_dot_ref',
    'e',
    'e_dot',
    'P_hat',
    'V_hat',
    'c_p',
    'c_v',
    'verbal_P_tracking_term',
    'verbal_D_tracking_term',
    'verbal_PD_tracking_term',
    'U_v',
    'physical_P_tracking_term',
    'physical_D_tracking_term',
    'physical_PD_tracking_term',
    'U_p',
    'phrase',
    'U',
    'human_disturbance',
    'system_dynamics_input',
]
DATA_COLLECTION_SAMPLING_RATE = 200

# Real time data plotting
PLOTTING_SAMPLING_RATE = 30

# Robot
ROBOT_IP = '169.254.9.43'
FORCE_EWMA_TAU = 0.5
TRANSLATIONAL_FORCE_DEADBAND = 2

# Experiment

This baseline is made up of all componentes: Physical PD, Physical Admittance, Verbal PD, Verbal Admittance, F2L, Compliance Estimator, and $J$ Optimization Method.

In [ ]:
from timer import Timer
from trajectory_planner import TrajectoryPlanner
from model import SVMKNNModel
import pickle
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from tabular_data_store import TabularDataStore
from plot_client import PlotClient
from robot import Robot
from console_command_thread import ConsoleCommandThread
import scipy.stats
from phrase_mapping import PHRASE_MAPPING, MEAN_WORDS_PER_PHRASE
from camera_feed import CameraFeed
from rgbd_stream import RGBDStream_iOS
from hand_tracker import HandTracker
import random

#############
### SETUP ###
#############

### Experiment ###
experiment_timer = Timer()

### Trajectory Planner ###
traj = TrajectoryPlanner(REFERENCE_TRAJECTORY_CSV_PATH, INIT_POSE[:3])

### F2L ###
with open(F2L_MODEL_PATH, 'rb') as file:
    F2L_model: SVMKNNModel = pickle.load(file)
phrase = ''
voc = Vocalizer()
speaking_timer = Timer()

### Real time data collection ###
table = TabularDataStore(column_names=TABLE_COLUMN_NAMES)
table_timer = Timer()

### Real time data plotting ###
plot = PlotClient()

plot.create_plot("force_plot", title="Force vs Time", xlabel="Time (s)", ylabel="Force")
plot.create_line("force_plot", "Fx", color="r", label="Fx")
plot.create_line("force_plot", "Fy", color="g", label="Fy")
plot.create_line("force_plot", "Fz", color="b", label="Fz")

plot.create_plot("velocity_plot", title="Velocity vs Time", xlabel="Time (s)", ylabel="Velocity")
plot.create_line("velocity_plot", "vx", color="r", label="vx")
plot.create_line("velocity_plot", "vy", color="g", label="vy")
plot.create_line("velocity_plot", "vz", color="b", label="vz")

plot.create_plot("error_plot", title="Error vs Time", xlabel="Time (s)", ylabel="Error")
plot.create_line("error_plot", "ex", color="r", label="ex")
plot.create_line("error_plot", "ey", color="g", label="ey")
plot.create_line("error_plot", "ez", color="b", label="ez")

plot.create_plot("compliance_plot", title="Estimated Compliance vs Time", xlabel="Time (s)", ylabel="Probability")
plot.create_line("compliance_plot", "P_hat", color="orange", label="P̂ (Physical)")
plot.create_line("compliance_plot", "V_hat", color="purple", label="V̂ (Verbal)")
 
plot.create_plot("control_plot", title="Control Inputs vs Time", xlabel="Time (s)", ylabel="Control Norm")
plot.create_line("control_plot", "U_p", color="cyan", label="‖U_p‖")
plot.create_line("control_plot", "U_v", color="magenta", label="‖U_v‖")

plot.create_plot("U_v_projection_plot", title="Projected U_v Profile", xlabel="Future Steps", ylabel="U_v")
plot.create_line("U_v_projection_plot", "U_v_x", color="r", label="U_v,x")
plot.create_line("U_v_projection_plot", "U_v_y", color="g", label="U_v,y")
plot.create_line("U_v_projection_plot", "U_v_z", color="b", label="U_v,z")

plot_timer = Timer()

stream = RGBDStream_iOS()
camera_feed = CameraFeed("Language Only Experiment", stream, np.load("calibration_matrix.npy"))
hand_tracker = HandTracker()

experiment_timer.reset()
speaking_timer.reset()

x_dot = np.zeros(3)

while True:
    t = experiment_timer.t()
    dt = experiment_timer.dt()

    hand_tracker.update(camera_feed.frame)
    x = hand_tracker.get_position()
    x_dot = x_dot * 0.9 + hand_tracker.get_velocity() * 0.1

    ### Trajectory Planner ###
    traj.update_reference_trajectory(x)
    x_ref, x_dot_ref = traj.get_closest_target(x)

    ### Tracking errors ###
    e = x_ref - x
    e_dot = x_dot_ref - x_dot

    ### Verbal PD ###
    verbal_P_tracking_term = K_v * e
    verbal_D_tracking_term = B_v * e_dot
    verbal_PD_tracking_term = verbal_P_tracking_term + verbal_D_tracking_term

    ### Verbal Admittance ###
    U_v = verbal_PD_tracking_term

    ### F2L ###
    x_sim = np.copy(x)
    x_dot_sim = np.copy(x_dot)

    U_v_profile = []

    for _ in range(N_proj):
        x_ref_proj, x_dot_ref_proj = traj.get_closest_target(x_sim)
        
        e_proj = x_ref_proj - x_sim
        e_dot_proj = x_dot_ref_proj - x_dot_sim

        U_v_proj = K_v * e_proj + B_v * e_dot_proj
        U_v_profile.append(U_v_proj)

        U_proj = U_v_proj

        a_proj = (U_proj - B * x_dot_sim) / M
        x_dot_sim += a_proj * dt_proj
        x_sim += x_dot_sim * dt_proj

    U_v_profile = np.array(U_v_profile)
    impulse_curve = U_v_profile.cumsum(axis=0) * dt_proj
    final_impuse = impulse_curve[-1, :]

    speaking_period = MEAN_WORDS_PER_PHRASE / (0.36 * 2.2 ** (2 - 0.0 - 0.0))

    if not voc.is_uttering():
        if np.linalg.norm(final_impuse) < FINAL_IMPULSE_THRESHOLD or speaking_timer.t() <= speaking_period:
            phrase = ''
        else:
            words = F2L_model.force_to_phrase(impulse_curve[None, :])[0]
            phrase = PHRASE_MAPPING[' '.join(words).strip()]
            if np.linalg.norm(e) < 2.5 and np.linalg.norm(e_dot) < 0.05:
                phrase = random.choice(['good job'])
            speaking_timer.reset()

    did_utter = voc.utter(phrase)

    is_speaking = voc.is_uttering()

    ### Real time data collection ###
    if table_timer.t() >= DATA_COLLECTION_SAMPLING_RATE**(-1):
        table.append_row((
            t,
            x,
            x_dot,
            np.nan,
            x_ref,
            x_dot_ref,
            e,
            e_dot,
            np.nan,
            np.nan,
            np.nan,
            np.nan,
            verbal_P_tracking_term,
            verbal_D_tracking_term,
            verbal_PD_tracking_term,
            U_v,
            np.nan,
            np.nan,
            np.nan,
            np.nan,
            phrase,
            np.nan,
            np.nan,
            np.nan,
        ))
        
        table_timer.reset()
        
    ### Real time data plotting ###
    # if plot_timer.t() >= PLOTTING_SAMPLING_RATE**(-1):
    #     # plot.update_line("force_plot", "Fx", (t, F_h[0]))
    #     # plot.update_line("force_plot", "Fy", (t, F_h[1]))
    #     # plot.update_line("force_plot", "Fz", (t, F_h[2]))

    #     plot.update_line("velocity_plot", "vx", (t, x_dot[0]))
    #     plot.update_line("velocity_plot", "vy", (t, x_dot[1]))
    #     plot.update_line("velocity_plot", "vz", (t, x_dot[2]))

    #     plot.update_line("error_plot", "ex", (t, e[0]))
    #     plot.update_line("error_plot", "ey", (t, e[1]))
    #     plot.update_line("error_plot", "ez", (t, e[2]))

    #     # plot.update_line("compliance_plot", "P_hat", (t, P_hat))
    #     # plot.update_line("compliance_plot", "V_hat", (t, V_hat))

    #     # plot.update_line("control_plot", "U_p", (t, np.linalg.norm(U_p)))
    #     plot.update_line("control_plot", "U_v", (t, np.linalg.norm(U_v)))

    #     timesteps = np.arange(len(U_v_profile)) * dt_proj

    #     plot.update_line("U_v_projection_plot", "proj_v_impulse_x", (timesteps, impulse_curve[:, 0]), mode="replace")
    #     plot.update_line("U_v_projection_plot", "proj_v_impulse_y", (timesteps, impulse_curve[:, 1]), mode="replace")
    #     plot.update_line("U_v_projection_plot", "proj_v_impulse_z", (timesteps, impulse_curve[:, 2]), mode="replace")

    #     plot_timer.reset()

    camera_feed.draw_world_point(x, radius=max(1, int(5.0 / np.linalg.norm(camera_feed.frame.camera.position - x))), color=(255, 0, 0))
    camera_feed.draw_world_arrow(x, x + x_dot, thickness=5, color=(255, 0, 0))
    camera_feed.draw_world_point(x_ref, radius=max(1, int(5.0 / np.linalg.norm(camera_feed.frame.camera.position - x_ref))), color=(0, 255, 0))
    # for point in traj.positions:
    #     camera_feed.draw_world_point(point, radius=max(1, int(5.0 / np.linalg.norm(camera_feed.frame.camera.position - point))), color=(0, 255, 0))
    camera_feed.update_window()


# table.to_pandas().to_pickle(f'../data/experiments/physical_only/{USER_ID}.pkl')